<a href="https://colab.research.google.com/github/NickDeMiceli20/emotionlogic/blob/main/AudioConverter_LVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install ffmpeg
!pip install yt-dlp
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
import yt_dlp
import requests
import json
from time import sleep
import pandas as pd
import subprocess
import os


ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
    'ffmpeg-location': 'C:\PATH_Programs',
    'outtmpl': "./%(id)s.%(ext)s",
}

VIDEO_URL = "https://www.youtube.com/watch?v=PMneW1ZxHSc"

In [61]:
def save_audio(VIDEO_URL):
    _id = VIDEO_URL.strip()

    def get_vid(_id):
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            return ydl.extract_info(_id)

    # download the audio of the YouTube video locally
    meta = get_vid(_id)
    save_location = meta['id'] + ".mp3"

    # print('Saved mp3 to', save_location)

    return save_location


def audio_upload(save_location):
    HEADERS_AUTH_ONLY = {'authorization':  "fb1821f8185546b492d6cee272b88408"}
    UPLOAD_ENDPOINT = "https://api.assemblyai.com/v2/upload"
    CHUNK_SIZE = 5242880

    def read_file(filename):
        with open(filename, 'rb') as _file:
            while True:
                data = _file.read(CHUNK_SIZE)
                if not data:
                    break
                yield data

    upload_response = requests.post(
        UPLOAD_ENDPOINT,
        headers=HEADERS_AUTH_ONLY,
        data=read_file(save_location)
    )

    audio_url = upload_response.json()['upload_url']

    return audio_url


def transcription_analytics(audio_url):
    HEADERS = {
        'authorization':  "fb1821f8185546b492d6cee272b88408",
        'content-type': 'application/json'
    }
    TRANSCRIPT_ENDPOINT = "https://api.assemblyai.com/v2/transcript"

    data = {
        'audio_url': audio_url,
        'punctuate': True,
        'speaker_labels': True,
        'sentiment_analysis': True
    }

    transcription_response = requests.post(
        TRANSCRIPT_ENDPOINT, json=data, headers=HEADERS)
    print(transcription_response)
    # print(f'Transcription debug... {transcription_response.json()}')

    transcript_id = transcription_response.json()['id']
    polling_endpoint = TRANSCRIPT_ENDPOINT + '/' + transcript_id

    print("Transcribing at", polling_endpoint)

    status = 'submitted'
    while status != 'completed':
        print("Transcription in progress...")
        sleep(1)
        polling_response = requests.get(polling_endpoint, headers=HEADERS)
        transcript = polling_response.json()['text']
        status = polling_response.json()['status']

    print('Compiling transcript...')

    response_json = polling_response.json()

    # Extract text where speaker is 'B'
    # Pair up start and end values of speaker B
    sbf = []
  
    prev_end = 0
    sentence_start = 0

    # Define a threshold in seconds. If the gap between two words is larger than this, we consider it as a new sentence
    threshold = 2

    for segment in response_json['words']:
        if segment['speaker'] == 'B':
            start = segment['start'] / 1000
            end = segment['end'] / 1000
            

            # If the gap between the current start and the previous end is larger than threshold, it's a new sentence
            if start - prev_end > threshold:
                # Add the previous sentence to sbf
                if sentence_start != 0 and prev_end != 0:
                    sbf.append((sentence_start, prev_end))
                
                # Start a new sentence
                sentence_start = start

            # Update prev_end
            prev_end = end

    # Add the last sentence
    if sentence_start != 0 and prev_end != 0:
        sbf.append((sentence_start, prev_end))

    
    print(sbf)
    # Convert the mp3 to wav for segment extraction
    mp3_file = save_location
    wav_file = save_location.replace('.mp3', '.wav')
    subprocess.run(["ffmpeg", "-i", mp3_file, wav_file])

    # Now, extract segments from the wav file
   # Now, extract segments from the wav file
    output_files = []
    for i, (start, end) in enumerate(sbf):
        output_file = f"output{i}.wav"
        command = ["ffmpeg", "-i", wav_file, "-ss", str(start), "-to", str(end), "-c", "copy", output_file]
        subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output_files.append(output_file)

    # Create a text file with all the files to be concatenated
    with open("files.txt", "w") as f:
        for output_file in output_files:
            f.write(f"file '{output_file}'\n")

    # Now, concatenate all the output files
    final_output_file = "final_output.wav"
    command = ["ffmpeg", "-f", "concat", "-safe", "0", "-i", "files.txt", "-c", "copy", final_output_file]
    subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Finally, remove all the temporary output files
    for output_file in output_files:
        os.remove(output_file)

    os.remove("files.txt")

    iab_categories_result = response_json['iab_categories_result']['summary']
    filtered_categories = []
    for category, score in iab_categories_result.items():
        if score >= 0.70:
            last_tag = category.split(">")[-1]
            last_tag = last_tag.replace("&", " & ")
            last_tag = " ".join([word.capitalize()
                                 for word in last_tag.split()])
            filtered_categories.append(last_tag)

    categories = filtered_categories

    sentiment_analysis_results = response_json['sentiment_analysis_results']
    sentiment = json.dumps(sentiment_analysis_results)


save_location = save_audio(VIDEO_URL)
audio_url = audio_upload(save_location)
transcription_analytics(audio_url)

[youtube] Extracting URL: https://www.youtube.com/watch?v=PMneW1ZxHSc
[youtube] PMneW1ZxHSc: Downloading webpage
[youtube] PMneW1ZxHSc: Downloading android player API JSON
[info] PMneW1ZxHSc: Downloading 1 format(s): 251
[dashsegments] Total fragments: 1
[download] Destination: ./PMneW1ZxHSc.webm
[download] 100% of    1.89MiB in 00:00:00 at 4.95MiB/s               
[ExtractAudio] Destination: ./PMneW1ZxHSc.mp3
Deleting original file ./PMneW1ZxHSc.webm (pass -k to keep)
<Response [200]>
Transcribing at https://api.assemblyai.com/v2/transcript/66rcb81auv-ac44-49d7-9786-f63fd813abfc
Transcription in progress...
Transcription in progress...
Transcription in progress...
Transcription in progress...
Transcription in progress...
Transcription in progress...
Transcription in progress...
Transcription in progress...
Transcription in progress...
Transcription in progress...
Transcription in progress...
Transcription in progress...
Transcription in progress...
Transcription in progress...
Transcr